---
### Sample Solutions for Temperature Forecast Project
1. **Baseline Model**
2. **FB Prophet**

---

### 1. Making a simple, baseline model
- Only by using monthly means of temperature differences
    - i.e. only by decomposing trend and seasonality.
- Cons:
    - Does not model time dependence.
    - Does not model any remainder (i.e. noise / volatility).
- Pros:
    - Simple reconstruction of trend and seasonality; fairly explainable!

- **DISCLAIMER:** notebook is based off code written by a former spiced student. 
    - Please avoid copying this explicitly (or publishing it online), but merely use it as a reference / borrow some ideas or snippets from it, if you wish.

---

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import r2_score

#### a. Read data from Berkeley Earth

In [2]:
##### BERLIN TEMPELHOF
data_raw = pd.read_csv('155194-TAVG-Data.txt', sep='\s+', comment='%', header=None, skiprows=526, skipfooter=10)
data_raw.columns = pd.read_csv('155194-TAVG-Data.txt', nrows=0, skiprows=96, skipinitialspace=True).columns
data_raw.rename(columns={data_raw.columns[0] : data_raw.columns[0].replace('%','').strip()}, inplace=True)

<ipython-input-2-17f15c916647>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  data_raw = pd.read_csv('155194-TAVG-Data.txt', sep='\s+', comment='%', header=None, skiprows=526, skipfooter=10)


#### b. Clean / Augment Data

In [ ]:
data_raw['Date'] = pd.to_datetime(dict(year=data_raw['Year'], month=data_raw['Month'], day=1))
data_raw['Temperature'] = data_raw['Temperature'].fillna(method='bfill')
data_raw = data_raw[['Date', 'Year', 'Month', 'Temperature']]

In [ ]:
data_raw.head()

#### c. Split Data

In [ ]:
#Use the last 120 data points (=10 years) for testing.
y_train = data_raw.iloc[:-120].reset_index(drop=True) #Jan 1756 - Dec 2002
y_test = data_raw.iloc[-120:].reset_index(drop=True) # Jan 2003 - Dec 2012

In [ ]:
y_train.tail()

#### d. Remove the trend (difference the temperature column)

In [ ]:
#Apply to training
y_train['Temp_diff'] = y_train['Temperature'].diff().fillna(0.0)


In [ ]:
y_train.tail(3)

#### e. Extract the Seasonality (from the de-trended data)

In [ ]:
y_train['month_means'] = y_train.groupby('Month')['Temp_diff'].transform('mean')

In [ ]:
y_train.tail(3)

#### f. Re-construct the time series by taking the cumulative product of the monthly means.
- However, the trick is to make sure that the cumulative sum is based off the original temperature value, so the scale makes sense.

In [ ]:
y0_train = y_train.iloc[0]['Temperature']

In [ ]:
y0_train

In [ ]:
y_train.loc[0,'month_means'] = y0_train 

#replaces the first point with the real value, so the cumulative sum can be built off of this.
y_train['Prediction'] = np.cumsum(y_train['month_means'])

In [ ]:
y_train.head()

### g. Apply the same technique to the testing data set!
- The key, however, is that you use the data (month_means) that were learned from the training data!

In [ ]:
#Do the same to testing
y_test['month_means'] = y_train.loc[0:120,'month_means']
#use the monthly means from the training data, however!

y_test.loc[0,'month_means'] = y_test.loc[0,'Temperature']
#replaces the first point with the real value, so the cumulative sum can be built off of this.

y_test['Prediction'] = np.cumsum(y_test['month_means'])

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot_date(y_test['Date'],y_test['Temperature'],'r-')
plt.plot_date(y_test['Date'],y_test['Prediction'],'b-')
plt.legend(['Actual Temperature', 'Naive Forecast'])

print(f"Test Score: {r2_score(y_test['Temperature'], y_test['Prediction'])}")

#### h. Extending this model / technique into the future:
- Now that we have a proof of concept that we can replicate the behavior of the trend / seasonality pretty well, let's apply it to the full data set and extend it out into the future.
- Let's predict 30 years into the future
- Create a **"future dataframe"**
    - **KEY**: Make the first day of the *future dataframe* the last day of our *known dataset* (i.e. y_test)

In [ ]:
yrs = 30
y_future = pd.DataFrame(index = pd.date_range(start='Dec 2012', periods = 12*yrs, freq='MS'),
             data=[0]*12*yrs,
             columns=['month_means'])

In [ ]:
y_future 
#at this point, y_future only contains 1 column (month_means), which is just filled in with zeros

#### i. Make the first value of the future dataset the same as the last value in the known dataset
- Our known dataset (y_test) ends in **December 2012**
- The future dataset (y_future) starts on **December 2012**

In [ ]:
#the trick here is that the first value of the future dataset is the temperature of the last day in ytest
y_future.iloc[0, 0] = y_test.iloc[-1]['Temperature']

In [ ]:
y_future.head()

#### j. Add in the monthly means that were originally calculated on the training data set!
- This is the trickiest part!
    - Really need to be careful that you're lining up the values for the correct months.
    - Remember, we're using the grouped averages of the monthly temperature DIFFERENCES, so everything is shifted.

In [ ]:
mean_training_vals = y_train.groupby('Month')['Temp_diff'].transform('mean')
#the grouped averages of the monthly temp differences, just repeated until the end of the data series

In [ ]:
#Insert mean_training_vals starting on the second row onwards. Since the second row happens to be January, it fits right in.
y_future.iloc[1:] = mean_training_vals[0:(12*yrs)-1].values.reshape(-1,1)

In [ ]:
y_future.head()

#### k. Again, use the cumulative sum to "extrapolate" / predict into the future!

In [ ]:
#Use cumulative sum to predict into future
y_future['Prediction'] = np.cumsum(y_future['month_means'])

y_future['Prediction'].plot()

---

Note that the prediction seems to be going down. 
- This naive approach seems to entirely depend on which year you start using for training.
- For example:

In [ ]:
data_raw.set_index('Date')['2010':]['Temperature'].diff().mean()

In [ ]:
data_raw.set_index('Date')['1980':]['Temperature'].diff().mean()

In [ ]:
data_raw.set_index('Date')['1950':]['Temperature'].diff().mean()

In [ ]:
data_raw.set_index('Date')['1900':]['Temperature'].diff().mean()

---

#### Wrapping everything together!

##### Let's try:

In [ ]:
#### Try tweaking these values!! 
#### Depending on how far back in time you take the monthly averages, the trend will change!
yrs = [20, 50, 100]
train_begin = [1970, 1800, 1900]
####

for yr, begin_yr in zip(yrs, train_begin):
    y_future = pd.DataFrame(index = pd.date_range(start='Dec 2012', periods = 12*yr, freq='M'),
                 data=[0]*12*yr,
                 columns=['month_means'])

    #the trick here is that the first value of the future dataset is the temperature of the last day in ytest
    y_future.iloc[0, 0] = y_test.iloc[-1]['Temperature']

    mean_training_vals = y_train.set_index('Date')[f'{begin_yr}':].groupby('Month')['Temp_diff'].transform('mean')
    y_future.iloc[1:] = mean_training_vals[0:(12*yr)-1].values.reshape(-1,1)

    #Use cumulative sum to predict into future
    y_future['Prediction'] = np.cumsum(y_future['month_means'])

    plt.figure(figsize=(12,8))
    y_future['Prediction'].plot(legend=['prediction'])
    data_raw.set_index('Date')['Temperature'].plot(legend=['historical data'])
    plt.title(f'Temperature prediction {yr} years into the future, training model back to {begin_yr}.')


---
---
---

### 2. Using an out-of-the-box, "black-boxy" model 
- FB Prophet (Developed by Facebook)
- Installation: 
    - ``pip install fbprophet``, or
    - ``conda install -c conda-forge fbprophet``, or
    - In case that doesn't work, see installation documentation: https://facebook.github.io/prophet/docs/installation.html#python
        - As of August 2020, there seems to be some strange issues with importing after the conda install:
            - https://github.com/facebook/prophet/issues/1300

![text](https://s.dou.ua/img/announces/prophet-840.png)

---

- Cons:
    - Not easy to understand the math behind it.
    - There **IS** a published paper, which is only 12 pages long ([read it!](https://peerj.com/preprints/3190.pdf)) but it's not super easy to understand.
        - Also see: [The Math of Prophet: Breaking down the Equation behind Facebook’s open-source Time Series Forecasting procedure](https://medium.com/future-vision/the-math-of-prophet-46864fa9c55a)
    - Installation tends to be a bit more tedious / unreliable than it should be.
- Pros:
    - Great Documentation.
    - SUPER EASY to use.
    - Models non-linear trends, fit with yearly, weekly, and daily seasonality, PLUS holiday effects.
        - "*It works best with time series that have strong seasonal effects and several seasons of historical data. Prophet is robust to missing data and shifts in the trend, and typically handles outliers well.*"

See Youtube Video: [Forecasting at Scale: How and Why We Developed Prophet for Forecasting at Facebook](https://www.youtube.com/watch?v=pOYAXv15r3A)

---

#### a. Prerequisite:
- FB Prophet is a univariate, time series model.
- in order for it to work, you must have a pandas DataFrame with 2 columns:
    - 1st column must be of the Pandas DateTime type and must be called ``ds``
    - 2nd column must be some numerical variable you're trying to forecast, and must be called ``y``

In [ ]:
### Let's just do it on the raw, original data (without train-test-split)
df = data_raw[['Date', 'Temperature']]

In [ ]:
df = df.rename(columns={'Date':'ds', 'Temperature':'y'})

In [ ]:
df.head()

#### b. Import model
- *If this step even works for you, congratulations, cause it took me 3 attempts to get the installation working*

In [ ]:
from fbprophet import Prophet

In [ ]:
m = Prophet(weekly_seasonality=True, daily_seasonality=True)

#### c. Train model

In [ ]:
m.fit(df)

*Similar to a generalized additive model (GAM), with time as a regressor, Prophet fits several linear and non-linear functions of time as components. In its simplest form:*

``y(t) = g(t) + s(t) + h(t) + e(t)``

*where:*

- ``g(t)`` -> trend models non-periodic changes (i.e. growth over time)

    - You can also program in "saturated growth", e.g. try setting ``growth='logistic'``
    
- ``s(t)`` -> seasonality presents periodic changes (i.e. weekly, monthly, yearly)

    - *Prophet relies on Fourier series to provide a malleable model of periodic effects.*
    
- ``h(t)`` -> ties in effects of holidays (on potentially irregular schedules ≥ 1 day(s))

- ``e(t)`` -> covers idiosyncratic changes not accommodated by the model


*Prophet is essentially "**framing the forecasting problem as a curve-fitting exercise**" rather than looking explicitly at the time based dependence of each observation.*

#### d. Prepare "future dataframe"

In [ ]:
future = m.make_future_dataframe(periods=240, freq='MS', include_history=False)
#20 years into the future

In [ ]:
future.tail()

#### e. Predict

In [ ]:
forecast = m.predict(future)

In [ ]:
forecast.head()

#### f. Admire

In [ ]:
fig = m.plot(forecast)

In [ ]:
fig2 = m.plot_components(forecast)